# Investigating Fandango Movie Ratings

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Investigating-Fandango-Movie-Ratings" data-toc-modified-id="Investigating-Fandango-Movie-Ratings-1">Investigating Fandango Movie Ratings</a></span><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1.1">Introduction</a></span></li><li><span><a href="#Data-Exploration" data-toc-modified-id="Data-Exploration-1.2">Data Exploration</a></span></li><li><span><a href="#Goal-Adjustment" data-toc-modified-id="Goal-Adjustment-1.3">Goal Adjustment</a></span></li><li><span><a href="#Sample-Validation" data-toc-modified-id="Sample-Validation-1.4">Sample Validation</a></span><ul class="toc-item"><li><span><a href="#Minimum-Metecritic-User-Ratings-Count" data-toc-modified-id="Minimum-Metecritic-User-Ratings-Count-1.4.1">Minimum Metecritic User Ratings Count</a></span></li></ul></li></ul></li></ul></div>

## Introduction

This analysis we will be looking at movie rating data from an online movie ticket sales and ratings aggregator, [Fandango](https://www.fandango.com/movie-reviews). In October of 2015, Walt Hickey, a data journalist, found [strong evidence](https://fivethirtyeight.com/features/fandango-movies-ratings/) that Fandango's rating system was biased and dishonest. 

Hickey found that there's a significant discrepancy between the number of stars displayed to users and the actual rating, which he was able to find in the HTML of the page. He was able to find that:
* The actual rating was almost always rounded up to the nearest half-star. For instance, a 4.1 movie would be rounded off to 4.5 stars, not to 4 stars, as you may expect.
* In the case of 8% of the ratings analyzed, the rounding up was done to the nearest whole star. For instance, a 4.5 rating would be rounded off to 5 stars.
* For one movie rating, the rounding off was completely bizarre: from a rating of 4 in the HTML of the page to a displayed rating of 5 stars.

Our task is to validate whether these findings are still true using more recent data from 2016 and 2017.

## Data Exploration

We have two sets of data:
* The original dataset that Hickey used in his 2015 investigation, available on [GitHub](https://github.com/fivethirtyeight/data/tree/master/fandango).
* A separate collection of ratings from 2016 and 2017, also available on [GitHub](https://github.com/mircealex/Movie_ratings_2016_17).

In [1]:
import pandas as pd
import numpy as np

In [2]:
# read in our datasets
movies_2015 = pd.read_csv('fandango_score_comparison.csv')
movies_2016_17 = pd.read_csv('movie_ratings_16_17.csv')

In [3]:
movies_2015.head()

,FILM,RottenTomatoes,RottenTomatoes_User,Metacritic,Metacritic_User,IMDB,Fandango_Stars,Fandango_Ratingvalue,RT_norm,RT_user_norm,...,IMDB_norm,RT_norm_round,RT_user_norm_round,Metacritic_norm_round,Metacritic_user_norm_round,IMDB_norm_round,Metacritic_user_vote_count,IMDB_user_vote_count,Fandango_votes,Fandango_Difference
0,Avengers: Age of Ultron (2015),74,86,66,7.1,7.8,5.0,4.5,3.70,4.3,...,3.90,3.5,4.5,3.5,3.5,4.0,1330,271107,14846,0.5
1,Cinderella (2015),85,80,67,7.5,7.1,5.0,4.5,4.25,4.0,...,3.55,4.5,4.0,3.5,4.0,3.5,249,65709,12640,0.5
2,Ant-Man (2015),80,90,64,8.1,7.8,5.0,4.5,4.00,4.5,...,3.90,4.0,4.5,3.0,4.0,4.0,627,103660,12055,0.5
3,Do You Believe? (2015),18,84,22,4.7,5.4,5.0,4.5,0.90,4.2,...,2.70,1.0,4.0,1.0,2.5,2.5,31,3136,1793,0.5
4,Hot Tub Time Machine 2 (2015),14,28,29,3.4,5.1,3.5,3.0,0.70,1.4,...,2.55,0.5,1.5,1.5,1.5,2.5,88,19560,1021,0.5


In [4]:
movies_2016_17.head()

,movie,year,metascore,imdb,tmeter,audience,fandango,n_metascore,n_imdb,n_tmeter,n_audience,nr_metascore,nr_imdb,nr_tmeter,nr_audience
0,10 Cloverfield Lane,2016,76,7.2,90,79,3.5,3.80,3.60,4.50,3.95,4.0,3.5,4.5,4.0
1,13 Hours,2016,48,7.3,50,83,4.5,2.40,3.65,2.50,4.15,2.5,3.5,2.5,4.0
2,A Cure for Wellness,2016,47,6.6,40,47,3.0,2.35,3.30,2.00,2.35,2.5,3.5,2.0,2.5
3,A Dog's Purpose,2017,43,5.2,33,76,4.5,2.15,2.60,1.65,3.80,2.0,2.5,1.5,4.0
4,A Hologram for the King,2016,58,6.1,70,57,3.0,2.90,3.05,3.50,2.85,3.0,3.0,3.5,3.0


In [5]:
movies_2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   FILM                        146 non-null    object 
 1   RottenTomatoes              146 non-null    int64  
 2   RottenTomatoes_User         146 non-null    int64  
 3   Metacritic                  146 non-null    int64  
 4   Metacritic_User             146 non-null    float64
 5   IMDB                        146 non-null    float64
 6   Fandango_Stars              146 non-null    float64
 7   Fandango_Ratingvalue        146 non-null    float64
 8   RT_norm                     146 non-null    float64
 9   RT_user_norm                146 non-null    float64
 10  Metacritic_norm             146 non-null    float64
 11  Metacritic_user_nom         146 non-null    float64
 12  IMDB_norm                   146 non-null    float64
 13  RT_norm_round               146 non

In [6]:
movies_2016_17.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   movie         214 non-null    object 
 1   year          214 non-null    int64  
 2   metascore     214 non-null    int64  
 3   imdb          214 non-null    float64
 4   tmeter        214 non-null    int64  
 5   audience      214 non-null    int64  
 6   fandango      214 non-null    float64
 7   n_metascore   214 non-null    float64
 8   n_imdb        214 non-null    float64
 9   n_tmeter      214 non-null    float64
 10  n_audience    214 non-null    float64
 11  nr_metascore  214 non-null    float64
 12  nr_imdb       214 non-null    float64
 13  nr_tmeter     214 non-null    float64
 14  nr_audience   214 non-null    float64
dtypes: float64(10), int64(4), object(1)
memory usage: 25.2+ KB


There are 146 rows of data and 22 columns in Hickey's 2015 dataset, and 214 rows of data and 15 columns in the 2016-17 dataset. Both datasets are complete, with no null values to consider, and a combination of float, integer, and string objects. 

Columns immediately relevant to our analysis:
*  __FILM__, __Fandango_Stars__, __Fandango_Ratingvalue__, __Fandango_votes__, and __Fandango_Difference__ from the movies_2015 dataframe
* __movie__, __year__, and __fandango__ from the movies_2016_17 dataframe

The methodology used for data collection are as follows: 
* The movies_2015 dataframe's [read_me](https://github.com/fivethirtyeight/data/blob/master/fandango/README.md) states that it contains every film that has a Rotten Tomatoes rating, a RT User rating, a Metacritic score, a Metacritic User score, and IMDb score, and at least 30 fan reviews on Fandango. 
* The 2016-2017 data's [read_me](https://github.com/mircealex/Movie_ratings_2016_17/blob/master/README.md) states the dataset contains movie ratings data for 214 of the most popular movies (with a significant number of votes) released in 2016 and 2017. As of March 22, 2017, the ratings were up to date. Significant changes should be expected mostly for movies released in 2017.

The collection process for both of these sets were not random samples, as movies on the site did not have equal chance for selection.

## Goal Adjustment

We have determined that there are several problems that we will need to navigate around:
* Neither of the datasets are random samples, meaning we may have a samples that are unrepresentative of the population.
* The datasets have different sampling methodology, meaning we can't accurately narrow the window of our analysis to only look at popular movies.
* Fandango no longer posts their own Fandango ratings, and only posts Rotten Tomato Scores, meaning we won't be able to accurately resample our data.

The best we can do is to try and validate our second dataset to determine if the 2016-2017 movies can fall into the 'popular' category. Because the 2015 data determined that popular movies are those with at least 30 Fandango reviews, Fandango reviews being a metric that the 2016-2017 dataset does not have and one we can not find ourselves, we will use the Metacritic user vote counts as a proxy. First we will determine the minimum count of Metacritic user reviews from the first dataset and compare whether a sample of the movies in the 2016 dataset are relatively close to or above that number. If they are, we can assume the second dataset is fairly comparable to the second relative to movie popularity.

Our goal will be then to recreate the distributions we saw in Hickey's analysis to see if the 2015 left skew  persists in the 2016-2017 Fandango rating dataset.

## Sample Validation

### Minimum Metecritic User Ratings Count 

Let's start by determining what the minimum values of each dataset to see if they have a comparable floor using Metacritic as the proxy.

In [9]:
meta_min_2015 = movies_2015['Metacritic_user_vote_count'].min()
meta_mean_2015 = movies_2015['Metacritic_user_vote_count'].mean()
print(meta_min_2015)
print(meta_mean_2015)

4
185.7054794520548
